## Install Redis Client
- https://pypi.org/project/redis/

In [1]:
!pip3 install redis

     |████████████████████████████████| 173 kB 5.9 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.10/bin/python3.10 -m pip install --upgrade pip' command.


## Redis docker 
```bash
docker run --name redis -e REDIS_PORT_NUMBER=7000 -e ALLOW_EMPTY_PASSWORD=yes -p 7000:7000 bitnami/redis:latest
```

In [6]:
import redis
r = redis.Redis(host='127.0.0.1', port=7000, db=0)

In [7]:
r.set('foo', 'bar')

True

In [8]:
r.get('foo')

b'bar'

## Chapter 3

In [9]:
import time 
import threading

def notrans():
    print(r.incr('notrans:'))
    time.sleep(0.1)
    r.incr('notrans:', -1)
    
if 1:
    for i in range(3):
        threading.Thread(target=notrans).start()
    time.sleep(0.5)

1
23



In [10]:
def trans():
    pipeline = r.pipeline()
    pipeline.incr('trans:')
    time.sleep(0.1)
    pipeline.incr('trans:', -1)
    print(pipeline.execute()[0])
    
if 1:
    for i in range(3):
        threading.Thread(target=trans).start()
    time.sleep(0.5)

111


